# To create the extracting files for barotropic tide (for 5 days each)

These files will filter and extract the barotropic tide from the ssh_dedac files in "**/bettik/bellemva/MITgcm/MITgcm_filtered_final**"

In [2]:
import numpy as np

In [ ]:
date = np.arange(np.datetime64("2012-05-01"),np.datetime64("2012-08-01"),np.timedelta64(2,'D'))

In [ ]:
date

Firstly this cell copys the file extract_tide.py into the number of submitted jobs. 

In [ ]:
# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()


for d in date : 
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")

    command = "cp extract_tide.py extract_tide_" + d.astype('str').replace("-","") + ".py"
    
    
    command += "\n"

    file.write(command)

    file.close()


For the missing dates : 

In [ ]:
# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()


for i in range (0,219,5) :

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")

    command = "cp extract_tide_missing_date.py extract_tide_missing_date" + str(i) + ".py"
    
    
    command += "\n"

    file.write(command)

    file.close()



Then, this cell updates the the line 28 to update the variable **init_date** that defines the initial date of the submitted job.

In [ ]:
for d in date :

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/extract_tide_"+d.astype('str').replace("-","")+".py","r")

    data = file.readlines()

    data[28] = "init_date = np.datetime64(\""+d.astype('str')+"\")\n"
    #data[137] = "    highpass_IT = highpass(lambda_bar,nx,ny,wavnum2D)\n"
    #data[0] = "print(\"job launched\") \n"
    #data[1] = "import os \n"

    file.close()

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/extract_tide_"+d.astype('str').replace("-","")+".py","w")

    file.writelines(data)

    file.close()

And for the missing dates : 

In [ ]:
for i in range (0,219,5) :

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/extract_tide_missing_date"+str(i)+".py","r")

    data = file.readlines()

    data[28] = "missing_date = np.load(\"./missing_date.npy\")["+str(i)+":"+str(i+5)+"]\n"
    #data[137] = "    highpass_IT = highpass(lambda_bar,nx,ny,wavnum2D)\n"
    #data[0] = "print(\"job launched\") \n"
    #data[1] = "import os \n"

    file.close()

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/extract_tide_missing_date"+str(i)+".py","w")

    file.writelines(data)

    file.close()

This cell below creates the submission file with extension .ksh in order to launch the extract_tide.py files 

In [ ]:
for d in date : 

    date_str = d.astype('str').replace("-","")

    file = open("/home/bellemva/CCS/submission_files/extract_tide/lambda_600/submission_bar_"+date_str+".sh","w")

    file.write("#!/bin/bash \n")
    file.write("#OAR -n submission_bar_"+date_str+"\n")
    file.write("#OAR -l /nodes=1/core=2,walltime=16:00:00 \n")
    file.write("#OAR --stdout submission_bar_"+date_str+".out \n")
    file.write("#OAR --stderr submission_bar_"+date_str+".err \n")
    file.write("#OAR --project data-ocean \n \n \n")

    file.write("source /home/bellemva/miniconda3/etc/profile.d/conda.sh \n")
    file.write("conda activate pangeo-forge \n")
    file.write("cd /home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/ \n")
    file.write("python extract_tide_"+date_str+".py")

    file.close()

    file = open("/home/bellemva/CCS/submission_files/extract_tide/lambda_600/file.sh","a")

    file.write("oarsub -S /home/bellemva/CCS/submission_files/extract_tide/lambda_600/submission_bar_"+date_str+".sh \n")

    file.close()



For missing dates : 

In [ ]:
for i in range (0,219,5) :

    #date_str = d.astype('str').replace("-","")

    file = open("/home/bellemva/CCS/submission_files/extract_tide/lambda_600/submission_bar_missing_date"+str(i)+".sh","w")

    file.write("#!/bin/bash \n")
    file.write("#OAR -n submission_bar_missing_date"+str(i)+"\n")
    file.write("#OAR -l /nodes=1/core=2,walltime=03:00:00 \n")
    file.write("#OAR --stdout submission_bar_missing_date"+str(i)+".out \n")
    file.write("#OAR --stderr submission_bar_missing_date"+str(i)+".err \n")
    file.write("#OAR --project data-ocean \n \n \n")

    file.write("source /home/bellemva/miniconda3/etc/profile.d/conda.sh \n")
    file.write("conda activate pangeo-forge \n")
    file.write("cd /home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/ \n")
    file.write("python extract_tide_missing_date"+str(i)+".py")

    file.close()

    file = open("/home/bellemva/CCS/submission_files/extract_tide/lambda_600/file.sh","a")

    file.write("oarsub -S /home/bellemva/CCS/submission_files/extract_tide/lambda_600/submission_bar_missing_date"+str(i)+".sh \n")

    file.close()



### 2 . Joining the hourly MITgcm_bar files into daily ones 

This cell change the time unit of each of the files into "**hours since 2012-01-01 00:00:00**"

In [5]:
#dates = np.arange(np.datetime64("2012-04-01"),np.datetime64("2012-10-31"),np.timedelta64(1,"h"))
dates = np.arange(np.datetime64("2012-04-01"),np.datetime64("2012-10-31"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncatted -h -O -a units,time,o,c,\"hours since 2012-01-01 00:00:00\" "

    #date_split = d.astype('str').split("T")

    #command += "/bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc \n"
    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()

    

This cell updates the time coordinate : 

In [6]:
#dates = np.arange(np.datetime64("2012-05-01"),np.datetime64("2012-08-01"),np.timedelta64(1,"h"))
dates = np.arange(np.datetime64("2012-04-01"),np.datetime64("2012-10-31"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncap2 -h -O -s "

    command += "time+="+str((d-np.datetime64("2012-01-01")).astype('int')*24)

    #date_split = d.astype('str').split("T")

    #command += " /bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc "
    command += " /bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "

    #command += " /bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc \n"
    command += " /bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()

This cell transforms the time coordinate into a record coordinate : 

In [3]:
dates = np.arange(np.datetime64("2012-05-01"),np.datetime64("2012-08-01"),np.timedelta64(1,"h"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncks -h -O --mk_rec_dmn time "

    date_split = d.astype('str').split("T")

    command += "/bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/MITgcm_bar_"+date_split[0].replace("-","")+"T"+str(int(date_split[1]))+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()

This cell creates the bash file that will concatenate the hourly barotropic tide datasets into daily ones. Then execute **file.sh**.

In [4]:
dates = np.arange(np.datetime64("2012-05-01"),np.datetime64("2012-08-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncrcat -h "

    for i in range (24) : 

        command += "/bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/MITgcm_bar_"+d.astype('str').replace("-","")+"T"+str(i)+".nc "
    
    command += "/bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/daily_files/MITgcm_bar_"+d.astype('str').replace("-","")+".nc \n"

    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()

    

This cell updates the variable **ssh_bar** into **ssh_bar_600** 

In [ ]:
dates = np.arange(np.datetime64("2012-05-01"),np.datetime64("2012-08-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncrename -h -O -v ssh_bar,ssh_bar_600 "

    command += "/bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/daily_files/MITgcm_bar_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/daily_files/MITgcm_bar_"+d.astype('str').replace("-","")+".nc \n"

    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()

    

The cell adds the variable $ssh\_bar\_600$ to the **MITgcm_filt** files. It uses the nco command $ncks$ Then execute **file.sh**.

In [ ]:
dates = np.arange(np.datetime64("2012-05-01"),np.datetime64("2012-08-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncks -h -A "

    command += "/bettik/bellemva/MITgcm/MITgcm_bar/lambda_bar_600/daily_files/MITgcm_bar_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()

In [3]:
dates = np.arange(np.datetime64("2012-06-01"),np.datetime64("2012-07-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncks -h -A "

    command += "/bettik/bellemva/MITgcm/MITgcm_it/MITgcm_it_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()

These cells create the variables **$ssh\_corr=ssh\_dedac-ssh\_bar$** and **$ssh\_igw=ssh\_hf-ssh\_bar$**

In [3]:
dates = np.arange(np.datetime64("2012-06-01"),np.datetime64("2012-07-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncap2 -h -A -s \"ssh_cor=ssh_dedac-ssh_bar\" "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    command += "ncap2 -h -A -s \"ssh_igw=ssh_hf-ssh_bar\" "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()


This cell deletes the variables **$ssh\_corr$**, **$ssh\_igw$** and **$ssh\_bar\_600$**.

In [2]:
dates = np.arange(np.datetime64("2012-06-01"),np.datetime64("2012-07-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncks -h -C -O -x -v ssh_cor,ssh_igw,ssh_bar_600,ssh_it "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()


This cell deletes global attributes **history_of_appended_files** and **history**.

In [6]:
dates = np.arange(np.datetime64("2012-04-01"),np.datetime64("2012-10-31"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncatted -h -a history,global,d,, "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    command += "ncatted -h -a history_of_appended_files,global,d,, "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"
    
    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()

This cell adds the variable **mask** of the file **mask.nc** to all the files. 

In [9]:
dates = np.arange(np.datetime64("2012-04-01"),np.datetime64("2012-10-31"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    command = "ncks -h -A -v mask /bettik/bellemva/MITgcm/mask/mask.nc "
    
    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()


This cell adds NaN to all pixels where **mask=False**. It aims at redifining the coastline more accurately. 

In [16]:
dates = np.arange(np.datetime64("2012-04-01"),np.datetime64("2012-10-31"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","w")
file.close()

for d in dates:

    #command = "ncap2 -h -O -s 'where(mask==1) ssh_bar_600=ssh_bar_600.get_miss()' " 
    
    #command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "
    #command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    command = "ncap2 -h -O -s 'where(mask==1) ssh_bm=ssh_bm.get_miss()' "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "
    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    command += "ncap2 -h -O -s 'where(mask==1) ssh_dedac=ssh_dedac.get_miss()' "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "
    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    command += "ncap2 -h -O -s 'where(mask==1) ssh_hf=ssh_hf.get_miss()' "

    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "
    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file.sh","a")
    
    file.write(command)

    file.close()



In [ ]:
dates = np.arange(np.datetime64("2012-05-01"),np.datetime64("2012-08-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file2.sh","w")
file.close()

for d in dates:

    command = "ncap2 -h -O -s 'where(mask==1) ssh_bar_600=ssh_bar_600.get_miss()' " 
    
    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc "
    command += "/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_"+d.astype('str').replace("-","")+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/lambda_bar_600/file2.sh","a")
    
    file.write(command)

    file.close()

This cell selects the CCS zone and saves it in the **/bettik/bellemva/2022a_CCS_CalVal** repertory. 

In [ ]:
dates = np.arange(np.datetime64("2012-06-01"),np.datetime64("2012-09-01"))

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()




### Modifying file name 

In [ ]:
missing_date = np.load("./missing_date.npy")

# deleting the content of file.sh 
file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","w")
file.close()

for d in missing_date : 

    day_str = d.astype('str').split("T")[0]
    time_index = d.astype('str').split("T")[1]
    time_index_bis = str(int(d.astype('str').split("T")[1]))

    command = "mv /bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+day_str.replace("-","")+"T"+time_index+"T"+time_index_bis+".nc "
    command+= "/bettik/bellemva/MITgcm/MITgcm_bar/MITgcm_bar_"+day_str.replace("-","")+"T"+time_index_bis+".nc \n"

    file = open("/home/bellemva/CCS/data/filter_MITgcm/extract_bar_tide/file.sh","a")
    
    file.write(command)

    file.close()

